<h1 align="left">Dataset Factory</h1>

<p align="left">
  ITESM
  
  <a href="https://www.linkedin.com/in/juanrtato/">Juan Ricardo Albarracin B.</a>
  <br>
  <a href="">Luis Ángel Oporto Añacato.</a>
  <br>
  <a href="">David Alexis García Espinosa.</a>
  <br>
  <b>Last updated:</b> <i>17/05/2025</i>
  <br><br>
  <a target="_blank">
    <img src="https://github.com/QData/TextAttack/workflows/Github%20PyTest/badge.svg" alt="Testing">
  </a>
  <a href="https://img.shields.io/badge/version-0.1.0-blue.svg?cacheSeconds=2592000">
    <img src="https://img.shields.io/badge/version-0.1.0-blue.svg?cacheSeconds=2592000" alt="Version" height="18">
  </a>
</p><br>

In [2]:
CONFIG_PATH = '../datalake/config_vtt.json'
MODEL_PATH = '../datalake/TSVIT/best.pth'
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts')))

In [3]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [4]:
import ast
import json
import torch
from tsvit import torch_utils, model_architecture
from visiontotext import visiontotextmodel
from pastis24 import get_dataloaders

/home/juanrtato/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

2.7.0+cu126
12.6
True


In [6]:
device_ids = [0]
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)
device = torch_utils.get_device(device_ids, allow_cpu=False)
with open("../datalake/label_names_en.json", "r") as json_file:
    label_names_en = json.load(json_file)
with open("../datalake/colormap.txt", "r") as txt_file:
    colormap = txt_file.readlines()
colormap = [ast.literal_eval(line.strip().rstrip(',')) for line in colormap]

In [7]:
records = []
dataloaders_vtt = get_dataloaders(config)

Loading PASTIS2SEQUENCE dataset...
PASTIS2SEQUENCE dataset loaded!


In [12]:
sample = next(iter(dataloaders_vtt['train']))
sample_dict, texts, img_path = sample
print("Sample input shape:", sample_dict['inputs'].shape)
print("Texts:", texts) # has the same length as sample_dict['inputs'] (batch size)
print("Image path:", img_path) # has the same length as sample_dict['inputs'] (batch size)

Sample input shape: torch.Size([24, 60, 24, 24, 11])
Texts: ('This area has been identified with the crop of soft winter wheat spanning approximately 45900 m2, winter rapeseed spanning approximately 8400 m2, which accounts for a considerable fraction of the total agricultural area: 57600 m2.\n', 'The crop of can be seen in the image leguminous fodder with a coverage of approximately 17500 m2, which covers a notable fraction of the total arable land: 57600 m2.\n', 'The image highlights the crop of grapevine with a coverage of approximately 43300 m2, winter durum wheat with a coverage of approximately 1300 m2, leguminous fodder with a coverage of approximately 100 m2, which equals a meaningful portion of the cultivated land in the region: 57600 m2.\n', 'No crop presence was recorded in this region.', 'No crop presence was recorded in this region.', 'The crop appearing in the image is soft winter wheat occupying around 14900 m2, winter barley covering about 1000 m2, which forms an essenti